In [1]:
import transformers
#2.8.0 original transformer version
print(transformers.__version__)

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from transformers import AutoModelForMaskedLM

4.4.2


## Preparing the dataset


In [2]:
#The original data for this was unfortunately overwritten so the results will vary from that in the paper
datasets = load_dataset("csv", data_files={"train": 'fine_tuning_training_data.csv', "validation": 'fine_tuning_test_data.csv'})


Using custom data configuration default-683cb3f965779f22
Reusing dataset csv (/sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [5]:
block_size=128

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [7]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
#     batch_size=1000,
    num_proc=4,
)

# masked language modeling



In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

ERROR! Session/line number was not unique in database. History logging moved to new session 2738


Loading cached processed dataset at /sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-0d8fca891f2fd12b.arrow
Loading cached processed dataset at /sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-4e2cc335b2dd382e.arrow
Loading cached processed dataset at /sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-904a30bb2c3e9040.arrow
Loading cached processed dataset at /sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-66d4153a478c9222.arrow
Loading cached processed dataset at /sailhome/katezhou/.cache/huggingface/datasets/csv/default-683cb3f965779f22/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
#     batch_size=16,
    num_proc=4,
)

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetunedtest",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
#     push_to_hub=False,
    save_strategy='epoch',
    num_train_epochs=4,
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()